In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [ ]:
# clonar repositorio 
! git clone https://github.com/fabrilopez/datasets.git

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pathlib

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications.vgg16 import VGG16

In [ ]:
tf.__version__

In [ ]:
data_dir = 'datasets/dataset_500'
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
label_names={'aca_315': 0, 'buck_guapo': 1, 'buck_meteoro': 2, 'klein_escorpion': 3}
label_names

In [ ]:
BATCH_SIZE = 64
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [ ]:
epochs = 50
init_lr = 0.0001
optimizer = tf.keras.optimizers.Adam(lr=init_lr, decay=init_lr / epochs)

In [ ]:
all_images = list(data_dir.glob('*/*'))
all_images = [str(path) for path in all_images]
random.shuffle(all_images)

all_labels=[label_names[pathlib.Path(path).parent.name] for path in all_images]

data_size = len(all_images)

train_test_split = (int)(data_size*0.2)

x_train = all_images[train_test_split:]
y_train = all_labels[train_test_split:]

x_validation = all_images[:train_test_split]
y_validation = all_labels[:train_test_split]


In [ ]:
len(x_train), len(y_train), len(x_validation), len(y_validation)

In [ ]:
def decode_img(x,y):
  image = tf.io.read_file(x)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_WIDTH, IMG_HEIGHT))
  return image,y

In [ ]:
def create_ds(x,y):
  ds = tf.data.Dataset.from_tensor_slices((x,y))
  ds = ds.map(decode_img)
  ds = ds.shuffle(buffer_size=len(x))
  ds = ds.repeat()
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

In [ ]:
train_ds = create_ds(x_train,y_train)
validation_ds=create_ds(x_validation,y_validation)

In [ ]:
train_ds, validation_ds

In [ ]:
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
VGG16_MODEL.trainable=False

In [ ]:
prediction_layer = tf.keras.layers.Dense(len(label_names),activation='softmax')

In [ ]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  Flatten(),
  Dense(4096, activation='relu'),
  prediction_layer
])

In [ ]:
VGG16_MODEL.summary()

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_ds, 
                    epochs=epochs, 
                    steps_per_epoch=8, 
                    validation_steps=8, 
                    validation_data=validation_ds)

In [ ]:
validation_steps = 30
 
loss0,accuracy0 = model.evaluate(validation_ds, steps = validation_steps)
 
print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

In [ ]:
# resumen para la exactitud
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# resumen para la funcion de pérdidas
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# hacer una prediccion
#image_name = 'datasets/seeds/kleinescorpion_0010.jpg'
#image_name = 'datasets/seeds/aca315_0010.jpg'
image_name = 'datasets/seeds/buckmeteoro_0010.jpg'
#image_name = 'datasets/seeds/buckguapo_0010.jpg'
image = tf.io.read_file(image_name)
image = tf.image.decode_jpeg(image, channels=3)
plt.imshow(image)
image = tf.cast(image, tf.float32)
image = (image/127.5) - 1
image = tf.image.resize(image, (IMG_WIDTH, IMG_HEIGHT))
image4d = tf.expand_dims(image,0)
prediction = model.predict(image4d)

In [ ]:
print(prediction)

In [ ]:
np_key = np.argmax(prediction)
for label, key in label_names.items():
  if key == np_key:
    print('La semilla seleccionada pertenece a la variedad: ' + label)